In [1]:
import pyro
import torch
import numpy as np
from pyro.infer.importance import Importance

# Whatsapp puzzle

Probability of a day being Sunday is givn by Bernoulli distribution
$$P(A) = p_{A}(x) = \mathrm{Bern}\left(x;1/7\right)$$

If the day is Sunday, number of messages $B$ per hour is given by the Poisson distribution
$$P(B|A=1) = p_{B}(x|A=1) = \mathrm{Poi}(x;3)$$
otherwise
$$P(B|A=0) = p_{B}(x|A=0) = \mathrm{Poi}(x;10)$$

If receives $4$ messages on a day, what is the probability that the day is Sunday
$$P(A=1|B=4)$$

# Calculate

In [2]:
def scale():
    weight = pyro.sample("weight", pyro.distributions.Uniform(0.0, 3.0), is_cont=False)
    if weight < 2.2:
        m = pyro.sample("measurement_1", pyro.distributions.Normal(weight - 1, 0.75), is_cont=False)
    else:
        tmp = pyro.sample("measurement_2", pyro.distributions.Normal(weight + 1, 0.75), is_cont=False)
        m = pyro.sample("measurement_1", pyro.distributions.Normal(tmp + 1, 0.25), is_cont=False)
    pyro.sample("obs", pyro.distributions.Normal(3.1, 0.25), obs=m)
    return weight.item()



In [9]:
importance = Importance(scale, guide=None, num_samples=20000)
importance.run()

In [11]:
log_weights = np.array([w.item() for w in importance.log_weights])
values = np.array([t.nodes['weight']['value'].item() for t in importance.exec_traces])
weights = np.exp(log_weights)
np.sum(weights * values) / np.sum(weights)

2.4446210441454292

# Sample from posterior via systematic resampling

In [25]:
import math
def systematic_resampling(log_weights, values):
    import torch

    mx = max(log_weights)
    weight_sum = sum(math.exp(log_weight - mx) for log_weight in log_weights)
    u_n = torch.distributions.Uniform(0, 1).sample().item()
    sum_acc = 0.0
    resamples = []
    for (log_weight, value) in zip(log_weights, values):
        weight = math.exp(log_weight - mx) * len(values) / weight_sum
        sum_acc += weight
        while u_n < sum_acc:
            u_n += 1
            resamples.append(value)
    return resamples


In [26]:
resamples = systematic_resampling(log_weights, values)                   
print(np.mean(resamples))                                            

2.445070327234268


In [6]:
from pyro.infer.importance import Importance
import pyro.infer.mcmc as pyromcmc
from pathlib import Path
import pyro
import torch
import numpy as np

kernel = pyromcmc.NPDHMC(
    scale,
    step_size=0.1,
    num_steps=50,
    adapt_step_size=False,
)
count = 1000
mcmc = pyromcmc.MCMC(kernel, num_samples=count, warmup_steps=count // 10)
mcmc.run()
samples = mcmc.get_samples()
for key, value in samples.items():
    print(key, value.shape, value.mean(0))

Sample: 100%|██████████| 1100/1100 [08:42,  2.11it/s, step size=1.00e-01, acc. prob=0.537]

weight torch.Size([1000]) tensor(2.2481, grad_fn=<MeanBackward1>)
measurement_2 torch.Size([660]) tensor(2.4618, grad_fn=<MeanBackward1>)
measurement_1 torch.Size([1000]) tensor(3.2792, grad_fn=<MeanBackward1>)


In [5]:

kernel = pyromcmc.HMC(
    scale,
    step_size=0.1,
    num_steps=50,
    adapt_step_size=False,
)
count = 1000
mcmc = pyromcmc.MCMC(kernel, num_samples=count, warmup_steps=count // 10)
mcmc.run()
samples = mcmc.get_samples()
for key, value in samples.items():
    print(key, value.shape, value.mean(0))

Sample: 100%|██████████| 1100/1100 [00:58, 18.70it/s, step size=1.00e-01, acc. prob=0.045]


measurement_1 torch.Size([1000]) tensor(3.2458, grad_fn=<MeanBackward1>)
weight torch.Size([1000]) tensor(2.1792, grad_fn=<MeanBackward1>)
